# Аналіз Рекомендаційної Системи на датасеті MovieLens

**Мета:** Побудувати та оцінити кілька моделей матричної факторизації за допомогою бібліотеки `surprise`, підібрати оптимальні параметри для SVD та обрати найкращий алгоритм на основі метрики RMSE.

**Кроки аналізу:**
1.  Завантаження даних.
2.  Оцінка базової моделі SVD.
3.  Підбір гіперпараметрів для SVD за допомогою GridSearchCV.
4.  Порівняння з алгоритмами SVD++ та NMF.
5.  Фінальні висновки.

In [1]:
# Крок 1: Імпорт необхідних бібліотек
import pandas as pd
from surprise import Reader, Dataset
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV

### Крок 2: Завантаження та Огляд Даних

**Навіть з відомим датасетом, швидко огляньмо його.**

In [2]:
# Крок 2: Завантаження та огляд даних
# Завантажуємо вбудований датасет movielens-100k. 
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\user/.surprise_data/ml-100k


In [6]:
# Для розуміння, подивимось на структуру даних.
# Ми можемо отримати доступ до сирих рейтингів.
raw_ratings = data.raw_ratings
print(f"Завантажено {len(raw_ratings)} рейтингів.")

Завантажено 100000 рейтингів.


In [8]:
# Подивимось на перші 5 елементів
print(raw_ratings[:5])

[('196', '242', 3.0, '881250949'), ('186', '302', 3.0, '891717742'), ('22', '377', 1.0, '878887116'), ('244', '51', 2.0, '880606923'), ('166', '346', 1.0, '886397596')]


**Згідно з документацією**
1. Перший елемент — це невелике число,схоже на ID користувача.
2. Другий — також схожий на ID (фільму).
3. Третій — це число від 1.0 до 5.0. Очевидно, це рейтинг.
4. Четвертий — це величезне ціле число. Це класичний вигляд Unix timestamp (кількість секунд з 1 січня 1970 року).

In [9]:
# Конвертуємо в Pandas DataFrame для кращого огляду перших 5 записів
df = pd.DataFrame(raw_ratings, columns=['UserID', 'ItemID', 'rating', 'timestamp'])
df.head()

,UserID,ItemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [10]:
# Крок 3: Побудова та Оцінка Базової Моделі (Baseline)
# Завжди встановлюємо базовий рівень (baseline). Це показник, який ми будемо намагатися перевершити
svd_baseline = SVD()
# Оцінюємо модель за допомогою 5-кратної крос-валідації
# verbose=True покаже процес для кожної з 5 частин даних
cv_results_svd_baseline = cross_validate(svd_baseline, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# Зберігаємо середнє значення для майбутнього порівняння
mean_rmse_svd_baseline = cv_results_svd_baseline['test_rmse'].mean()
print(f"Середній RMSE для базової SVD: {mean_rmse_svd_baseline:.4f}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9379  0.9363  0.9350  0.9364  0.9389  0.9369  0.0014  
MAE (testset)     0.7384  0.7382  0.7355  0.7394  0.7416  0.7386  0.0020  
Fit time          0.76    0.78    0.78    0.76    0.78    0.77    0.01    
Test time         0.22    0.13    0.08    0.12    0.09    0.13    0.05    
Середній RMSE для базової SVD: 0.9369



### Аналіз Помилки Базової Моделі (RMSE = 0.9369)

Давайте оцінимо, наскільки значущою є отримана помилка, з двох різних точок зору.

1.  **З точки зору Шкали Рейтингів:**
    *   **Шкала:** Наш датасет MovieLens використовує 5-бальну шкалу (1, 2, 3, 4, 5 зірок).
    *   **Інтерпретація:** Помилка в `0.9369` означає, що в середньому прогнози відхиляються від реальних оцінок приблизно на *майже одну повну зірку*. Це різниця між "хорошим фільмом" (4) і "середнім" (3).
    *   **Оцінка:** З точки зору користувацького досвіду, це відчутно, і є простір для покращення.

2.  **З точки зору Контексту Моделювання:**
    *   **"Найгірший" результат (Baseline):** Модель, яка завжди прогнозує середній рейтинг (близько 3.5), мала б значно вищу помилку (RMSE > 1.1). На тлі цього, наш результат `0.9369` — це значний крок вперед.
    *   **"Найкращий" результат (Людська точність):** Навіть просунуті системи рідко опускають RMSE значно нижче 0.85, оскільки смаки людей є суб'єктивними.

---

> **Висновок:** Результат **RMSE = 0.9369** є **хорошою та очікуваною стартовою точкою (baseline)**. Він значно кращий за примітивні підходи і знаходиться в межах стандартних показників для цього завдання.
   

In [12]:
# Крок 4: Підбір Найкращих Параметрів для SVD
# Використовуємо систематичні методи для підбору параметрів, такі як GridSearchCV.
# Визначаємо "сітку" параметрів, які хочемо перевірити
param_grid = {
    'n_epochs': [20, 25], # Кількість ітерацій (епох)
    'lr_all': [0.005, 0.01], # Швидкість навчання для всіх параметрів
    'reg_all': [0.02, 0.04] # Коефіцієнт регуляризації для всіх параметрів
}
# Створюємо об'єкт GridSearchCV
# cv=3 - 3-кратна крос-валідація
# n_jobs=-1 - використати всі доступні ядра процесора для розпаралелювання
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

# "Навчаємо" GridSearchCV на даних
gs.fit(data)

# --- Результати ---
print(f"{gs.best_score['rmse']:.4f}")

print("\nНайкраща комбінація параметрів:")
print(gs.best_params['rmse'])
# Зберігаємо найкращий результат для фінального порівняння
best_rmse_svd_tuned = gs.best_score['rmse']

0.9365

Найкраща комбінація параметрів:
{'n_epochs': 25, 'lr_all': 0.005, 'reg_all': 0.04}


In [14]:
# Порівнюємо нашу покращену модель з іншими релевантними підходами, використовуючи ті ж самі метрики та процедуру оцінки.
# Крок 5: Оцінка альтернативних моделей: SVD++ та NMF
# --- Оцінка SVD++ ---
svdpp = SVDpp()
cv_results_svdpp = cross_validate(svdpp, data, measures=['RMSE'], cv=5, verbose=True)
mean_rmse_svdpp = cv_results_svdpp['test_rmse'].mean()
print(f"Середній RMSE для SVD++: {mean_rmse_svdpp:.4f}")

# --- Оцінка NMF ---
nmf = NMF()
cv_results_nmf = cross_validate(nmf, data, measures=['RMSE'], cv=5, verbose=True)
mean_rmse_nmf = cv_results_nmf['test_rmse'].mean()
print(f"Середній RMSE для NMF: {mean_rmse_nmf:.4f}")

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9200  0.9103  0.9217  0.9248  0.9262  0.9206  0.0056  
Fit time          13.66   13.77   13.84   13.86   13.97   13.82   0.11    
Test time         2.57    2.62    2.56    2.55    2.53    2.57    0.03    
Середній RMSE для SVD++: 0.9206
Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9613  0.9744  0.9616  0.9668  0.9505  0.9629  0.0078  
Fit time          1.24    1.24    1.24    1.24    1.22    1.24    0.01    
Test time         0.13    0.07    0.07    0.07    0.13    0.09    0.03    
Середній RMSE для NMF: 0.9629


# Висновки

## Порівняння результатів

Для оцінки ефективності різних моделей рекомендаційних систем було проведено 5-кратну крос-валідацію, а для моделі SVD - додатково підібрано гіперпараметри за допомогою GridSearchCV. Результати за метрикою RMSE (чим нижче, тим краще) представлені нижче.

In [18]:
# Фінальне порівняння результатів
print("--- ФІНАЛЬНА ТАБЛИЦЯ РЕЗУЛЬТАТІВ ---")
print(f"1. Базова модель SVD:\t\tRMSE = {mean_rmse_svd_baseline:.4f}")
print(f"2. SVD з підбором параметрів:\tRMSE = {best_rmse_svd_tuned:.4f}")
print(f"3. Модель SVD++:\t\tRMSE = {mean_rmse_svdpp:.4f}")
print(f"4. Модель NMF:\t\t\tRMSE = {mean_rmse_nmf:.4f}")

--- ФІНАЛЬНА ТАБЛИЦЯ РЕЗУЛЬТАТІВ ---
1. Базова модель SVD:		RMSE = 0.9369
2. SVD з підбором параметрів:	RMSE = 0.9365
3. Модель SVD++:		RMSE = 0.9206
4. Модель NMF:			RMSE = 0.9629


# Висновки та Вибір Оптимальної Моделі

Згідно з результатами крос-валідації, **`SVD++` є найточнішою моделлю** з `RMSE = 0.9206`.

*   **Порівняння точності:** `SVD++` суттєво перевершує як базовий `SVD` (0.9369), так і `NMF` (0.9629).
*   **Ефект налаштування:** Підбір параметрів для `SVD` в даному експерименті не дав значного покращення (`0.9369` -> `0.9365`).
*   **Причина переваги SVD++:** Його вища точність пояснюється здатністю враховувати неявний зворотний зв'язок (факт оцінки фільму користувачем).

**Рекомендація:** Для досягнення максимальної точності слід обрати **`SVD++`**. Якщо ж пріоритетом є швидкість навчання, базовий `SVD` залишається хорошою, хоча й менш точною, альтернативою.